# Install necessary libraries

In [ ]:
# INSTALL numpy. COMMENT THIS IF numpy IS ALREADY INSTALLED.
# %pip install numpy

In [ ]:
# INSTALL tensorflow. COMMENT THIS IF tensorflow IS ALREADY INSTALLED.
# %pip install tensorflow

In [ ]:
# INSTALL tensorflow_hub. COMMENT THIS IF tensorflow_hub IS ALREADY INSTALLED.
# %pip install tensorflow_hub

In [ ]:
# INSTALL tqdm. COMMENT THIS IF tqdm IS ALREADY INSTALLED.
# %pip install tqdm

In [ ]:
# INSTALL torchvision. COMMENT THIS IF torchvision IS ALREADY INSTALLED.
# %pip install torchvision

In [ ]:
# INSTALL ipynb. COMMENT THIS IF ipynb IS ALREADY INSTALLED
# %pip install ipynb

In [ ]:
# INSTALL ipynb. COMMENT THIS IF ipynb IS ALREADY INSTALLED
# %pip install keras

In [ ]:
# IF YOU WANT TO CHECK ALL PACKAGES ARE INSTALLED SUCCESSFULLY.
# UNCOMMENT THE FOLLOWING LINE AND CHECK FOR INSTALLED PACKAGES.
# %pip list

In [ ]:
# INSTALL matplotlib. COMMENT THIS IF ipynb IS ALREADY INSTALLED
# %pip install matplotlib

# Import necessary libraries

In [ ]:
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"                                     # IGNORE THE WARNINGS FROM IMPORTING TENSERFLOW.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub                                                 # USING TENSORFLOW_HUB WE WILL HAVE A PRETRAINED EFFICIENT MODEL THAT WE WILL BE USING.
from tqdm import tqdm                                                        # JUST FOR GETTING A PROGRESS BAR FOR THE CURRENT EPOCH.                                                
from ipynb.fs.full.dataset import get_loaders                                # we need to import dataset.ipynb notebook.
import matplotlib.pyplot as plt                                              # IMPORT matplotlib to create graphs.

# Setting variables and environment (using EfficientNetB3 with 20 Epochs)

In [ ]:
physical_devices = tf.config.list_physical_devices( "GPU" )                  # SETTING THE PHYSICAL DEVICES.
if len( physical_devices ) > 0:                                              # MIGHT NOT NECESSARY, BUT ITS SAFE PRECAUSION TO TAKE.
    tf.config.experimental.set_memory_growth( physical_devices[0], True)

URL = "https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1"        # URL TO TENSORFLOW HUB WHERE DOWNLOAD THE FEATURE VECTOR ASSOCIATE TO AN EFFICIENTNET MODEL.
                                                                             # HERE WE ARE GOING TO USE THE SMALLEST EFFICIENTNET MODEL JUST TO MAKE IT BIT FASTER TO TRAIN.
                                                                             # THAT IS EFFICIENTNET B0 MODEL.
IMG_SIZE = 300                                                               # IMAGE SIZE DEPEND ON THE MODEL. IF INCREASE b0 TO b1, YOU NEED TO CHANGE THE IMAGE SIZE TO 240.
BATCH_SIZE = 32
NUM_CLASSES = 355
NUM_EPOCHS = 10
DATA_DIR = "data/"
MODEL_PATH = "efficientnetb0/"                                               # SETTING WHERE WE WANT TO SAVE THE MODEL.
LOAD_MODEL = False

os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "1"
os.environ["TFHUB_CACHE_DIR"] = "E:/SDGP_TROJANS/machine_learning_component/efficientNet_b3_2.91GB_data/TFHUB_CACHE"    # WHERE TO SAVE THE CACHE FILES WHILE BUILDING MODEL.

# Setting sequential model and building model

In [ ]:
def get_model( url, img_size, num_classes):
    model = tf.keras.Sequential([
        hub.KerasLayer( url, trainable = True ),
        layers.Dense( 1000, activation = "relu" ),
        layers.Dropout( 0.3 ),
        layers.Dense( num_classes, activation = "softmax" ),
    ])

    model.build( [None, img_size, img_size, 3] )                              # BUILDING THE MODEL. None FOR THE BATCH. 3 IS NUMBER OF CHANNELS.
    return model

In [ ]:
@tf.function                                                                  # WRAPPING WITH tf.function FOR SOME ADDITIONAL OPTIMIZATION.
def train_step( data, labels, acc_metric, model, loss_fn, optimizer ):
    
    with tf.GradientTape() as tape:
        predictions = model( data, training = True )
        loss = loss_fn( labels, predictions )

    gradients = tape.gradient( loss, model.trainable_weights )
    optimizer.apply_gradients( zip( gradients, model.trainable_weights ) )
    acc_metric.update_state( labels, predictions )                             # AT LAST UPDATING THE ACCURACY METRIC, WHICH WILL BE SORT OF RUNNING ACCURACY ACROSS THE EPOCHS.

# Model evaluation

In [ ]:
def evaluate_model( ds_validation, model ):
    accuracy_metric =  keras.metrics.SparseCategoricalAccuracy()               # HAVING accuracy_metric TO KEEP TRACK OF WHAT IS THE ACCURACY ACROSS THE VALIDATION SET.

    for idx, ( data, labels) in enumerate( ds_validation ):
        # CONVET THE DATA TO HAVE THE CORRECT DIMENSIONS.
        data = data.permute( 0, 2, 3, 1 )                
        data = tf.convert_to_tensor( np.array( data ) )
        labels = tf.convert_to_tensor( np.array( labels ) )

        y_pred = model( data, training = False )
        accuracy_metric.update_state( labels, y_pred )

    accuracy = accuracy_metric.result()
    print( f"Accuracy over validation set: { accuracy }" )
     

# Plotting (training and validation loss) and ( training and validation accuracy)

In [ ]:
def plotTheGraphs(train_loss, val_loss, train_acc, val_acc):
    # Plot training and validation loss
    plt.plot(train_loss, label='train')
    plt.plot(val_loss, label='val')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Plot training and validation accuracy
    plt.plot(train_acc, label='train')
    plt.plot(val_acc, label='val')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()


# Apply a learning rate scheduler
# Training Loop
# calculate train loss, train accuracy, validation loss, validation accuracy

In [ ]:
def main():
    train_loader, dev_loader = get_loaders( DATA_DIR + "train", DATA_DIR + "dev", BATCH_SIZE, IMG_SIZE )  
    
    # SETUP THE TRAINING.
    if LOAD_MODEL:
        print("Loading model")
        model = keras.models.load_model( MODEL_PATH )
    
    # IF WE WANT TO CONTINUE THE TRAINING, WE CAN START THE TRAINING AND RERUNNING IT. OTHERWISE, WE CAN USE THE CREATED get_model() function.
    else:
        print("Building model")
        model = get_model( URL, IMG_SIZE, NUM_CLASSES )

    # SETUP THE OPTIMIZER
    # SPECIFY THE LEARNING RATE(lr) AS 3e-4. 
    # WE ARE NOT ACTUALLY GOING TO PLAYing AROUND TOO MUCH HEAVY LEARNING RATE.
    # YOU CAN USE LEARNING RATE SCHEDULAR, THAT WOULD MOST LIKELY IMPROVE THE MODEL. 

    # USING LEARNING RATE SCHEDULER INSTEAD OF HARDCODING LEARNING RATE AS FOLLOWING COMMENTED LINE (e.g. optimizer = keras.optimizers.Adam( learning_rate = 3e-4 )).
    lr_schedule = keras.optimizers.schedules.ExponentialDecay( initial_learning_rate = 1e-3, decay_steps = 10000, decay_rate = 0.9 )
    optimizer = keras.optimizers.Adam(learning_rate = lr_schedule)

    # optimizer = keras.optimizers.Adam( learning_rate = 3e-4 )

    # CREATING LOSS FUNCTION.
    loss_fn = keras.losses.SparseCategoricalCrossentropy( from_logits = False )
    # THIS IS FOR THE RUNNING ACCURACY.
    acc_metric = keras.metrics.SparseCategoricalAccuracy()


    train_loss = []
    train_accuracy = []
    val_loss = []
    val_accuracy = []

    # TRAINING LOOP
    for epoch in range(NUM_EPOCHS):

        epoch_train_loss = []
        epoch_train_accuracy = []

        for idx, ( data, labels ) in enumerate( tqdm(train_loader) ):

            # SINCE WE ARE LOADING DATA IN PyTorch WE NEED TO SWAP THE ACCESS FOR THE NUMBER OF CHANNELS, BECAUSE THESE CHANNELS ARE NOT STRUCTURED IN THE SAMA WAY.
            # PyTorch: N X C X H X W       TensorFlow: N X H X W X C       N -      , H - HEight  , W - WIDTH  ,  C - CHANNELS
            # N IS IN THE SAME POSITION IN BOTH. H HAS 2 INDEX, GET H TO SECOND POSITION.LIKEWISE...

            data = data.permute( 0, 2, 3, 1 )
            data = tf.convert_to_tensor( np.array( data ) )
            labels = tf.convert_to_tensor( np.array( labels ) )

            train_step( data, labels, acc_metric, model, loss_fn, optimizer )

            # CHECKING THE INDEX
            if idx % 150 == 0 and idx > 0:
                # GET THE TRAINING ACCRACY, PRINT THAT ACCURACY. REMEMBER, THIS IS THE RUNNING TRAINING ACCURACY SO FAR IN THE EPOCHS.
                train_acc =  acc_metric.result()
                print(f"Accuracy over epoch (so far) {train_acc}" )
                
                # evaluation.
                evaluate_model( dev_loader, model )
                model.save( MODEL_PATH )


            # append loss and accuracy of the current epoch to relevant lists.
            epoch_train_loss.append( loss_fn(labels, model(data)).numpy() )
            epoch_train_accuracy.append( acc_metric.result().numpy() )

        # append the mean values of both epoch_train_loss and epoch_train_accuracy to the 
        train_loss.append( np.mean(epoch_train_loss) )
        train_accuracy.append( np.mean(epoch_train_accuracy) )


        epoch_val_loss = []
        epoch_val_accuracy = []

        for idx, (data, labels) in enumerate(dev_loader):

            # Swap the access for the number of channels to match TensorFlow
            data = data.permute(0, 2, 3, 1)
            data = tf.convert_to_tensor(np.array(data))
            labels = tf.convert_to_tensor(np.array(labels))

            # Predict on validation data
            epoch_val_loss.append( loss_fn(labels, model(data)).numpy() )
            acc_metric.update_state( labels, model(data) )
            epoch_val_accuracy.append( acc_metric.result().numpy() )


        val_loss.append( np.mean(epoch_val_loss) )
        val_accuracy.append( np.mean(epoch_val_accuracy) )

        # Print epoch summary
        print(  f"Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {epoch_train_loss[-1]:.4f}, Train Acc: {epoch_train_accuracy[-1]:.4f}, Val Loss: {epoch_val_loss[-1]:.4f}, Val Acc: {epoch_val_accuracy[-1]:.4f}" )

        # Reset the accuracy metric
        acc_metric.reset_states()

    # plot the graphs
    plotTheGraphs( train_loss, val_loss, train_accuracy, val_accuracy )

In [ ]:
if __name__ == "__main__":
    main()